In [ ]:
# 초기코드 - python
from keras.utils import np_utils
from keras.layers import Dense
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,ZeroPadding2D,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
#from PIL import Image
import os, glob, numpy as np
from keras.models import load_model
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Activation, Dense
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3 ,preprocess_input
import preprocess_crop


# 새 섹션

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/LPD_competition.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/968/32.jpg        
   creating: train/486/
  inflating: train/486/21.jpg        
  inflating: train/486/22.jpg        
  inflating: train/486/23.jpg        
  inflating: train/486/12.jpg        
  inflating: train/486/13.jpg        
  inflating: train/486/14.jpg        
  inflating: train/486/15.jpg        
  inflating: train/486/16.jpg        
  inflating: train/486/17.jpg        
  inflating: train/486/18.jpg        
  inflating: train/486/19.jpg        
  inflating: train/486/20.jpg        
  inflating: train/486/9.jpg         
  inflating: train/486/10.jpg        
  inflating: train/486/11.jpg        
  inflating: train/486/0.jpg         
  inflating: train/486/1.jpg         
  inflating: train/486/2.jpg         
  inflating: train/486/3.jpg         
  inflating: train/486/4.jpg         
  inflating: train/486/5.jpg         
  inflating: train/486/6.jpg         
  inflating: train/486/7.jpg         
  inflating: train/486/8.jpg

In [ ]:
train_data_dir = '/content/train'
img_height=299
img_width=299
batch_size=64

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.9,1.1),
    validation_split=0.2,
    #preprocessing_function=preprocess_input
    ) # set validation split

val_datagen = ImageDataGenerator(rescale=1./255,
                                 validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
#   interpolation='lanczos:center',
    interpolation='lanczos',
    shuffle=True,
    subset='training') # set as training data

validation_generator = val_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=False,
    subset='validation') # set as validation data

Found 39000 images belonging to 1000 classes.
Found 9000 images belonging to 1000 classes.


In [ ]:
incept = InceptionV3(include_top=False,input_shape=(299,299,3),weights='imagenet')
# for layer in model.layers[:-2]:
#     layer.trainable = False
model=Sequential()
model.add(incept)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(1000,activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              2049000   
Total params: 23,851,784
Trainable params: 23,817,352
Non-trainable params: 34,432
_________________________________________________________________


# 새 섹션

In [ ]:
#optimizer = SGD(lr=1e-4, momentum=0.9)
#optimizer RMSprop(lr=2e-5)
optimizer = keras.optimizers.Adam(lr=0.00005, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/inceptionv3_fine_tuning2.model'
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

In [ ]:
nb_epochs=20
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint,early_stopping]
    )
hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
!cp history.csv /content/gdrive/MyDrive/history.csv
!cp -r model /content/gdrive/MyDrive/model

Epoch 1/20
609/609 [==============================] - 1543s 3s/step - loss: 6.2994 - accuracy: 0.0952 - val_loss: 2.2825 - val_accuracy: 0.7779

Epoch 00001: val_loss improved from inf to 2.28247, saving model to ./model/inceptionv3_fine_tuning2.model
INFO:tensorflow:Assets written to: ./model/inceptionv3_fine_tuning2.model/assets
Epoch 2/20
 39/609 [>.............................] - ETA: 22:11 - loss: 2.8410 - accuracy: 0.7307

KeyboardInterrupt: ignored

In [ ]:
#또는
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

!cp history.json ./gdrive/MyDrive/_history.json
!cp history.csv ./gdrive/MyDrive/_history.csv

In [ ]:
!mkdir temp
!mv ./test/* ./temp
!mkdir ./test/unknown
!mv ./temp/* ./test/unknown/

In [ ]:
test_data_dir = '/content/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode=None, #결과 똑같이나온다 None이나 Categorical 이나
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=False
    )

In [ ]:
co=0
st=train_generator.class_indices.keys()
result_arr=list(st)
print(result_arr)

In [ ]:
# !rm -rf /content/test/unknown
# !mkdir /content/test/unknown

In [ ]:
model = load_model('/content/gdrive/MyDrive/model/model/inceptionv3_fine_tuing2.model')
prediction = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
predicted_class = np.argmax(prediction,axis=1)

In [ ]:
test = pd.read_csv('/content/sample.csv')
df = test.reindex(index=test.index[:72000], columns=list(test.columns))
print(df)

In [ ]:
co=0
for i in predicted_class :
    ans = i
    ans=result_arr[ans]
    ans=int(ans)
    fname =test_generator.filenames[co]
    fname = fname.replace("unknown/","")
    find_row=test.loc[(test['filename']==fname)]
    find_row=int(find_row.index.tolist()[0])
    df.loc[find_row,'prediction']=ans
    co+=1
    print(co)

In [ ]:
df.to_csv("result.csv",mode='w',index=False)
!cp result.csv ./gdrive/MyDrive/result.csv

# 새 섹션

# 새 섹션